Parseamos y estandarizamos los datos de los países de europa

In [ ]:
from utils.file_utils import parse_countries_data, standarize_data
import matplotlib.pyplot as plt
attributes, countries, dataset = parse_countries_data("ex_1/resources/europe.csv")
std = standarize_data(dataset)

import numpy as np

Parameters

In [ ]:
# grilla de salida es de dimensión output_dimension*output_dimension
output_dimension = int(np.floor(np.sqrt(std.shape[0]*0.5)))

#nro de epocas
epoch_limit = 1000

# tasa de aprendizaje inicial
initial_learning_rate = 1

# radio inicial
initial_radius = np.ceil(output_dimension*output_dimension/2)

# función de actualización de tasa de aprendizaje
update_learning_rate = (lambda e, k: initial_learning_rate*np.exp(-2*k/((epochs)*std.shape[0])))

#función de actualización de radio
update_radius = (lambda e, k: initial_radius*np.exp(-5*k/((epochs)*std.shape[0])))

In [ ]:
import networks.kohonen as kohonen
import numpy as np
kohonen_network = kohonen.KohonenNetwork(std.shape[1], output_dimension, 1)
epochs =[]
errors = []
def callback_fun(e, error):
    errors.append(error)
    epochs.append(e)
kohonen_network.train(std, countries, epoch_limit, callback_fun )


In [ ]:
# plot error vs epochs
plt.plot(epochs, errors)

In [ ]:

# plot radius values through iterations
plt.plot(range(len(kohonen_network.radius_values)), kohonen_network.radius_values)
plt.xlabel("Iteration")
plt.ylabel("Radius")

In [ ]:
#plot learning rate values through iterations
# print(kohonen_network.learning_rate_values)
plt.plot(range(len(kohonen_network.learning_rate_values)), kohonen_network.learning_rate_values)
plt.xlabel("Iteration")
plt.ylabel("Learning Rate")

In [ ]:
distances = np.zeros((output_dimension, output_dimension))
heatmap_data = np.zeros((output_dimension, output_dimension))
country_names = [['' for i in range(output_dimension)] for j in range(output_dimension)]
isolated_variables = np.zeros((len(std[0]),output_dimension, output_dimension))
for c,_input in enumerate(std):
    i, j, d = kohonen_network.get_activated_neuron(_input)
    distances[i][j] += d
    heatmap_data[i][j] += 1
    country_names[i][j]+=countries[c]
    for l in range(len(_input)):
        isolated_variables[l][i][j] += std[c][l]

quantization_error = np.sum(distances)/len(std)

for i in range(output_dimension):
    for j in range(output_dimension):
        distances[i][j] /= heatmap_data[i][j]
        for l in range(len(_input)):
            isolated_variables[l][i][j] /= heatmap_data[i][j]

print(quantization_error)


In [ ]:
import seaborn as sns

sns.heatmap(heatmap_data, annot=True)


In [ ]:
sns.heatmap(kohonen_network.u_matrix(), annot=True, cmap="Greys")

Plot isolated variables

In [ ]:
fig, ax = plt.subplots(len(std[0]),1, figsize=(8,40))
for i, var in enumerate(isolated_variables):
    ax[i].set_title(attributes[i])
    sns.heatmap(var, annot=True, ax=ax[i])
fig.tight_layout(pad=3.0)